### Долгодворова Мария, Домашнее задание № 1

### Импорт используемых библиотек

In [ ]:
# для удобного суммирования по спискам
import math
# для создания словарей и подсчета аминокислот
from collections import defaultdict, Counter
# для логарифмов
import numpy as np
# графики
import plotly.graph_objects as go
# работа с датафреймами
import pandas as pd

# Данные

### выбрана бактерия Chlamydia psittaci, скачаем CP002744.1

Для удобства полученный после запуска скрипта файл был добавлен на гитхаб

In [ ]:
! wget https://raw.githubusercontent.com/knapweedss/bioinfHSE/main/AlgoBioif/intergenes.fasta

--2024-04-18 13:49:01--  https://raw.githubusercontent.com/knapweedss/bioinfHSE/main/AlgoBioif/intergenes.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165092 (161K) [text/plain]
Saving to: ‘intergenes.fasta’

intergenes.fasta    100%[===================>] 161.22K  --.-KB/s    in 0.02s   

2024-04-18 13:49:01 (6.55 MB/s) - ‘intergenes.fasta’ saved [165092/165092]



In [ ]:
# распарсим файл
all_sequences = [] # наши последовательности
intergenes_cord = [] # координаты
with open ('intergenes.fasta', encoding='UTF-8') as f:
    File = f.readlines()
    for line in File[1::2]:
        all_sequences.append(line)
    for line in File[0::2]:
        intergenes_cord.append(line)

In [ ]:
# проверим, что нет последовательности длиной меньше 20
for s in all_sequences:
    if len(s) < 20:
        raise Exception('length is less than 20')
print('All sequences are ok')

All sequences are ok


In [ ]:
print('Всего последовательностей', len(all_sequences))

Всего последовательностей 688


Разобьем выборку на трейн и тест (по принципу четного и нечетного). Трейн для удобства будем хранить в строке, тк всегда работаем с частотностью на весь датасет, а тест сохраним в виде списка сток, так как в дальнейшем по такому будет удобнее считать вероятности для последовательности.

In [ ]:
train_ = ''
test_ = all_sequences[1::2]
for i in all_sequences[0::2]:
    for s in i:
        train_ += i

In [ ]:
train_ = train_.replace("\n", "") # чистим от символа переноса строки

# Бернуллиевская модель

In [ ]:
# считаем количество каждой буквы в трейне
train_res = Counter(train_)
train_res

Counter({'c': 11680583, 'a': 19028513, 'g': 11586123, 't': 18689865})

In [ ]:
# сколько всего в трейне
total_train = Counter(train_).total()
total_train

60985084

In [ ]:
prob_total = 0
probs = []
for i in train_res.values():
    res = i/total_train # получаем "вероятность нуклеотида"
    prob_total += (res) # для ассерта
    probs.append(np.log(res)) # логарифмируем и внесем в список вероятностей
if prob_total != 1:
    raise Exception('Sum of probabilities is not 1')

In [ ]:
probs

[-1.652701419144174,
 -1.1646907712064445,
 -1.6608212208030504,
 -1.1826479113547153]

In [ ]:
# смэппим нуклеотиды и их вероятности для удобной работы в дальнейшем
nucleotides = ['a', 'c', 't', 'g']
nucleotide_probabilities = dict(zip(nucleotides, probs))
nucleotide_probabilities

{'a': -1.652701419144174,
 'c': -1.1646907712064445,
 't': -1.6608212208030504,
 'g': -1.1826479113547153}

In [ ]:
test_probs = [] # вероятности последовательностей
for nucleo_seq in test_:
    pr = []
    for nucleo in nucleo_seq.replace("\n", ""):
        pr.append(nucleotide_probabilities[nucleo]) # получаем вероятность для буквы
    test_probs.append(sum(pr)) # суммируем (тк log) все вероятности для заданной последовательнрости

In [ ]:
test_probs[0:5] # посмотрим частично результат

[-85.64616540259783,
 -109.00216676141548,
 -35.863339106356406,
 -100.42948115459845,
 -294.46781093320595]

# Марковская модель 1-го порядка

In [ ]:
def calculate_markov_probability(sequence, amino, n):
    """
    Вычисляет логарифмические вероятности переходов для заданной аминокислоты
    с учетом контекста из предыдущих n аминокислот последовательности

    Params:
        sequence (str): последовательность аминокислот (train)
        amino (str): аминокислота, для которой рассчитывается вероятность
        n (int): количество учитываемых предыдущих аминокислот (размер контекста)

    Returns:
        dict: Словарь, где ключи — это строки из N предыдущих аминокислот,
                а значения — логарифмические вероятности
                того, что после данного контекста следует указанная аминокислота amino
    """
    transition_probabilities = defaultdict(lambda: defaultdict(int))
    total_transitions = defaultdict(int)

    # словарь частот переходов
    for i in range(len(sequence) - n):
        context = sequence[i:i+n]  # получаем последовательноость из N символов
        next_amino_acid = sequence[i + n]
        transition_probabilities[context][next_amino_acid] += 1
        total_transitions[context] += 1

    # считаем логарифмические вероятности переходов
    log_probability_a_following = {}
    for context, transitions in transition_probabilities.items():
        if amino in transitions and transitions[amino] > 0:
            probability = transitions[amino] / total_transitions[context]
            log_probability_a_following[context] = math.log(probability)
        else:
            # Чтобы избежать логарифма от нуля применяем логарифм к малому положительному числу
            log_probability_a_following[context] = math.log(1e-10)

    return log_probability_a_following

In [ ]:
amino = 'a'
probability_a = calculate_markov_probability(train_, amino, 1)
for amino_acid, probability in probability_a.items():
    print(f"Вероятность {amino} после {amino_acid} {probability:.2f}")

Вероятность a после c -1.19
Вероятность a после a -1.05
Вероятность a после g -1.09
Вероятность a после t -1.34


In [ ]:
amino = 'c'
probability_c = calculate_markov_probability(train_, amino, 1)
for amino_acid, probability in probability_c.items():
    print(f"Вероятность {amino} после {amino_acid} {probability:.2f}")

Вероятность c после c -1.55
Вероятность c после a -1.82
Вероятность c после g -1.62
Вероятность c после t -1.59


In [ ]:
amino = 'g'
probability_g = calculate_markov_probability(train_, amino, 1)
for amino_acid, probability in probability_g.items():
    print(f"Вероятность {amino} после {amino_acid} {probability:.2f}")

Вероятность g после c -1.96
Вероятность g после a -1.51
Вероятность g после g -1.56
Вероятность g после t -1.73


In [ ]:
amino = 't'
probability_t = calculate_markov_probability(train_, amino, 1)
for amino_acid, probability in probability_t.items():
    print(f"Вероятность {amino} после {amino_acid} {probability:.2f}")

Вероятность t после c -1.07
Вероятность t после a -1.32
Вероятность t после g -1.36
Вероятность t после t -1.04


In [ ]:
# для более удобной дальнейшей работы сделаем nested dict
markov_probs_1 = {
    'a': probability_a,
    't': probability_t,
    'g': probability_g,
    'c': probability_c
}
markov_probs_1

{'a': {'c': -1.1915796215452577,
  'a': -1.0460585658624055,
  'g': -1.0943409579146621,
  't': -1.335287690934545},
 't': {'c': -1.0707274400410611,
  'a': -1.3203292897918015,
  'g': -1.3607007439151277,
  't': -1.0359083779680902},
 'g': {'c': -1.9551902868029176,
  'a': -1.514898371063537,
  'g': -1.5606537571135985,
  't': -1.7289920910207022},
 'c': {'c': -1.5513469464497778,
  'a': -1.8213794646708985,
  'g': -1.6156566080944759,
  't': -1.58694045055996}}

In [ ]:
def get_probs(test, markov_probs, n, complement=False):
    """
    Вычисляет вероятности последовательностей

    Params:
        test (list): список последовательностей аминокислот (test)
        markov_probs (dict): вероятности
        n (int): длина контекста
        complement (bool):

    Returns:
        list: список вероятностей
    """
    test_probs = []
    for nucleo_seq in test:
        nucleo_seq = nucleo_seq.strip() # strip чтобы убрать '\n'
        s = n # счетчик для индексов
        pr = []
        if complement:
          nucleo_seq = nucleo_seq[::-1]
        for nucleo in nucleo_seq[n::]:
            pr.append(markov_probs[nucleo][nucleo_seq[s - n:s]])
            s+=1
        test_probs.append(sum(pr))
    return test_probs

In [ ]:
probs_mark_1 = get_probs(test_, markov_probs_1, 1)
probs_mark_1[0:5] # частичный вывод результата

[-69.64709166789996,
 -90.32138603451821,
 -30.309408512756484,
 -84.8395770077271,
 -264.895844865861]

# Марковская модель 2-го порядка

In [ ]:
# здесь уже не буду принтить вероятность каждой пары, довольно объемный вывод получается
context = 2
amino = 'a'
probability_a = calculate_markov_probability(train_, amino, context)

amino = 'c'
probability_c = calculate_markov_probability(train_, amino, context)

amino = 'g'
probability_g = calculate_markov_probability(train_, amino, context)

amino = 't'
probability_t = calculate_markov_probability(train_, amino, context)

In [ ]:
markov_probs_2 = {
    'a': probability_a,
    't': probability_t,
    'g': probability_g,
    'c': probability_c
}

In [ ]:
# посмотрим только вероятности для t, чтобы не засорять ноутбук принтами
markov_probs_2['t']

{'ca': -1.3331656514300427,
 'aa': -1.4555089444837426,
 'ag': -1.547429050351221,
 'gt': -0.9615191172103548,
 'tt': -0.9547404293540441,
 'ta': -1.1863956129808186,
 'ac': -1.2000835872410867,
 'at': -1.1841793632590902,
 'tg': -1.225294994189336,
 'cc': -0.9978763937174736,
 'cg': -1.2542238948831002,
 'tc': -1.015843461513019,
 'ct': -1.0586675039418831,
 'ga': -1.2735285965710796,
 'gg': -1.3420427771484955,
 'gc': -1.0834887783961458}

In [ ]:
# посчитаем вероятности на тесте
probs_mark_2 = get_probs(test_, markov_probs_2, 2)
probs_mark_2[0:5] # частичный вывод результата

[-67.02797496932334,
 -89.56719728390414,
 -28.277492616387583,
 -83.25920319569616,
 -265.21600712482586]

# Марковская модель 3-го порядка

In [ ]:
# здесь уже не буду принтить вероятность каждой пары, довольно объемный вывод получается
context = 3
amino = 'a'
probability_a = calculate_markov_probability(train_, amino, context)

amino = 'c'
probability_c = calculate_markov_probability(train_, amino, context)

amino = 'g'
probability_g = calculate_markov_probability(train_, amino, context)

amino = 't'
probability_t = calculate_markov_probability(train_, amino, context)

In [ ]:
markov_probs_3 = {
    'a': probability_a,
    't': probability_t,
    'g': probability_g,
    'c': probability_c
}

In [ ]:
# посмотрим только вероятность для t после ctg, чтобы не засорять ноутбук принтами
markov_probs_3['t']['ctg']

-1.2601539250057565

In [ ]:
# посчитаем вероятности на тесте
probs_mark_3 = get_probs(test_, markov_probs_3, context)
probs_mark_3[0:4] # частичный вывод результата

[-66.62984258476665,
 -88.1684285960955,
 -26.181739117159697,
 -79.78159490796176]

# Посмотрим на распределения вероятностей и разности суммарных логарифмов вероятностей последовательностей

In [ ]:
len(test_)

344

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Бернулли",
                                                        "Марковская модель 1-го порядка",
                                                        "Марковская модель 2-го порядка",
                                                        "Марковская модель 3-го порядка"))


bin_numb = len(test_)
fig.add_trace(go.Histogram(x=test_probs, nbinsx=bin_numb, marker=dict(color='cornflowerblue'),
                           name='Бернулли'), row=1, col=1)
fig.add_trace(go.Histogram(x=probs_mark_1, nbinsx=bin_numb, marker=dict(color='#00CC96'),
                           name='Марковская модель 1-го порядка'), row=1, col=2)
fig.add_trace(go.Histogram(x=probs_mark_2, nbinsx=bin_numb, marker=dict(color='#DDA0DD'),
                           name='Марковская модель 2-го порядка'), row=2, col=1)
fig.add_trace(go.Histogram(x=probs_mark_3, nbinsx=bin_numb, marker=dict(color='#FFF5EE'),
                           name='Марковская модель 3-го порядка'), row=2, col=2)


fig.update_layout(
    title='Распределения Логарифмических Вероятностей',
    bargap=0.2,
    template='plotly_dark',
    autosize=True,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [ ]:
def calculate_prob_diff(probs_markov_1, probs_markov_2, probs_markov_3, bernoulli_probs):
    # сделаем numpy массивы
    probs_markov_1 = np.array(probs_markov_1)
    probs_markov_2 = np.array(probs_markov_2)
    probs_markov_3 = np.array(probs_markov_3)
    bernoulli_probs = np.array(bernoulli_probs)

    # суммарные вероятности для каждой модели
    sum_markov_1 = np.sum(np.exp(probs_markov_1))
    sum_markov_2 = np.sum(np.exp(probs_markov_2))
    sum_markov_3 = np.sum(np.exp(probs_markov_3))
    sum_bernoulli = np.sum(np.exp(bernoulli_probs))

    # разность суммарных вероятностей для каждой пары моделей
    diff_markov_1_markov_2 = sum_markov_1 - sum_markov_2
    diff_markov_1_markov_3 = sum_markov_1 - sum_markov_3
    diff_markov_2_markov_3 = sum_markov_2 - sum_markov_3
    diff_bernoulli_markov_1 = sum_bernoulli - sum_markov_1
    diff_bernoulli_markov_2 = sum_bernoulli - sum_markov_2
    diff_bernoulli_markov_3 = sum_bernoulli - sum_markov_3

    return diff_markov_1_markov_2, diff_markov_1_markov_3, diff_markov_2_markov_3, diff_bernoulli_markov_1, diff_bernoulli_markov_2, diff_bernoulli_markov_3

In [ ]:
diff_markov_1_markov_2, diff_markov_1_markov_3, diff_markov_2_markov_3, diff_bernoulli_markov_1, diff_bernoulli_markov_2, diff_bernoulli_markov_3 = calculate_prob_diff(probs_mark_1, probs_mark_2, probs_mark_3, test_probs)

print("Разность суммарных вероятностей последовательностей между моделями:")
print(f"Марковская модель 1-го порядка и 2-го порядка: {diff_markov_1_markov_2}")
print(f"Марковская модель 1-го порядка и 3-го порядка: {diff_markov_1_markov_3}")
print(f"Марковская модель 2-го порядка и 3-го порядка: {diff_markov_2_markov_3}")
print(f"Бернулли и Марковская модель 1-го порядка : {diff_bernoulli_markov_1}")
print(f"Бернулли и Марковская модель 2-го порядка : {diff_bernoulli_markov_2}")
print(f"Бернулли и Марковская модель 3-го порядка : {diff_bernoulli_markov_3}")

Разность суммарных вероятностей последовательностей между моделями:
Марковская модель 1-го порядка и 2-го порядка: -7.451385596844643e-10
Марковская модель 1-го порядка и 3-го порядка: -3.977366998723986e-09
Марковская модель 2-го порядка и 3-го порядка: -3.2322284390395223e-09
Бернулли и Марковская модель 1-го порядка : -2.0246120504976867e-10
Бернулли и Марковская модель 2-го порядка : -9.47599764734233e-10
Бернулли и Марковская модель 3-го порядка : -4.179828203773755e-09


# Проверим на комплементарных последовательностях

Для марковских моделей (передаем флаг True, чтобы считалось по реверснутой последовательности)

In [ ]:
probs_mark_1_compl = get_probs(test_, markov_probs_1, 1, True)
probs_mark_1_compl[0:5]

[-69.52760113467062,
 -89.9774343272887,
 -30.470516844672275,
 -85.33608705036903,
 -264.9048995663811]

In [ ]:
probs_mark_2_compl = get_probs(test_, markov_probs_2, 2, True)
probs_mark_2_compl[0:5]

[-67.78200360104096,
 -88.4543268983937,
 -29.081550849255354,
 -82.30192784130966,
 -262.84673312275396]

In [ ]:
probs_mark_3_compl = get_probs(test_, markov_probs_3, 3, True)
probs_mark_3_compl[0:5]

[-67.41843664123355,
 -86.82654320941535,
 -28.385644064215196,
 -80.84918497863491,
 -263.47206154379455]

Для Бернулли:

Просто по первым значениям видим, что полученные вероятности схожи

In [ ]:
test_probs_compl = [] # вероятности последовательностей
for nucleo_seq in test_:
    pr = []
    for nucleo in nucleo_seq.replace("\n", "")[::-1]:
        pr.append(nucleotide_probabilities[nucleo]) # получаем вероятность для буквы
    test_probs_compl.append(sum(pr)) # суммируем (тк log) все вероятности для заданной последовательнрости
test_probs_compl[0:5]

[-85.6461654025979,
 -109.00216676141555,
 -35.8633391063564,
 -100.42948115459848,
 -294.46781093320567]

Чтобы убедиться, что вероятности для всех комплементарных последовательностей близки, можно сравнить, используя уже посчитанную разность суммарных вероятностей последовательностей между моделям

In [ ]:
diff_markov_1_markov_2_compl, diff_markov_1_markov_3_compl, diff_markov_2_markov_3_compl, diff_bernoulli_markov_1_compl, diff_bernoulli_markov_2_compl, diff_bernoulli_markov_3_compl = calculate_prob_diff(probs_mark_1_compl, probs_mark_2_compl, probs_mark_3_compl, test_probs_compl)

print("Разность суммарных вероятностей последовательностей между моделями:")
print(f"Марковская модель 1-го порядка и 2-го порядка - для некомплементарных: {diff_markov_1_markov_2}, для комплементарных: {diff_markov_1_markov_2_compl}")
print(f"Марковская модель 1-го порядка и 3-го порядка - для некомплементарных: {diff_markov_1_markov_3}, для комплементарных: {diff_markov_1_markov_3_compl}")
print(f"Марковская модель 2-го порядка и 3-го порядка: - для некомплементарных: {diff_markov_2_markov_3}, для комплементарных: {diff_markov_2_markov_3_compl}")
print(f"Бернулли и Марковская модель 1-го порядка - для некомплементарных: {diff_bernoulli_markov_1}, для комплементарных: {diff_bernoulli_markov_1_compl}")
print(f"Бернулли и Марковская модель 2-го порядка - для некомплементарных: {diff_bernoulli_markov_2}, для комплементарных: {diff_bernoulli_markov_2_compl}")
print(f"Бернулли и Марковская модель 3-го порядка - для некомплементарных: {diff_bernoulli_markov_3}, для комплементарных: {diff_bernoulli_markov_3_compl}")

Разность суммарных вероятностей последовательностей между моделями:
Марковская модель 1-го порядка и 2-го порядка - для некомплементарных: -7.451385596844643e-10, для комплементарных: -1.1131400207048082e-09
Марковская модель 1-го порядка и 3-го порядка - для некомплементарных: -3.977366998723986e-09, для комплементарных: -5.3334615187413116e-09
Марковская модель 2-го порядка и 3-го порядка: - для некомплементарных: -3.2322284390395223e-09, для комплементарных: -4.220321498036504e-09
Бернулли и Марковская модель 1-го порядка - для некомплементарных: -2.0246120504976867e-10, для комплементарных: -1.9301049400757028e-10
Бернулли и Марковская модель 2-го порядка - для некомплементарных: -9.47599764734233e-10, для комплементарных: -1.3061505147123785e-09
Бернулли и Марковская модель 3-го порядка - для некомплементарных: -4.179828203773755e-09, для комплементарных: -5.5264720127488815e-09


Видим, что небольшая разница есть, но в пределах допустимого, тк последовательностей много и есть наблюдается небольшой эффект накопления ошибки, но при сравнении отдельных значений вероятности для каждой последовательности разница минимальна

# AIC и BIC

# AIC = 2k - 2ln(L)

In [ ]:
L_bern = sum(test_probs)
L_bern

-103311.421566496

In [ ]:
L_mark_3 = sum(probs_mark_3)
L_mark_3

-89978.79736256467

In [ ]:
L_mark_2 = sum(probs_mark_2)
L_mark_2

-90423.81359774528

In [ ]:
L_mark_1 = sum(probs_mark_1)
L_mark_1

-91067.97740128118

In [ ]:
aic_bern = (2 * 4) - (2 * L_bern)
aic_mark_1 = (2 * 16) - (2 * L_mark_1)
aic_mark_2 = (2 * 64) - (2 * L_mark_2)
aic_mark_3 = (2 * 256) - (2 * L_mark_3)

In [ ]:
print('AIC Бернулли: ', aic_bern)
print('AIC Марковская модель 1-го порядка: ', aic_mark_1)
print('AIC Марковская модель 2-го порядка: ', aic_mark_2)
print('AIC Марковская модель 3-го порядка: ', aic_mark_3)

AIC Бернулли:  206630.843132992
AIC Марковская модель 1-го порядка:  182167.95480256237
AIC Марковская модель 2-го порядка:  180975.62719549055
AIC Марковская модель 3-го порядка:  180469.59472512934


Чем меньше AIC, тем лучше модель, судя по данной метрике лучше всего Марковская модель 3-го порядка, что в целом и логично, так как она учитывает больше контекста. Хотя она крайне близка по качеству к Марковской модели 2-го порядка.
Хуже всего работает Бернулли.

# BIC = ln(n) * k - 2ln(L)

In [ ]:
log_size = np.log(len(train_))

In [ ]:
bic_bernulli = (log_size * 4) - (2 * L_bern)
bic_markov1 = (log_size * 16) - (2 * L_mark_1)
bic_markov2 = (log_size * 64) - (2 * L_mark_2)
bic_markov3 = (log_size * 256) - (2 * L_mark_3)

In [ ]:
print('BIC Бернулли: ', bic_bernulli)
print('BIC Марковская модель 1-го порядка: ', bic_markov1)
print('BIC Марковская модель 2-го порядка: ', bic_markov2)
print('BIC Марковская модель 3-го порядка: ', bic_markov3)

BIC Бернулли:  206694.5476924626
BIC Марковская модель 1-го порядка:  182422.7730404447
BIC Марковская модель 2-го порядка:  181994.90014701991
BIC Марковская модель 3-го порядка:  184546.6865312468


Здесь лучше всего Марковская модель 2-го порядка, но опять все 3 модели показали приблизительно одинаковое качество, а хуже всего Бернулли

# Посмотрим на графики вероятностей на конкретных позициях

In [ ]:
dict_probs = {
    'Бернулли': test_probs,
    'Марковская модель 1-го порядка': probs_mark_1,
    'Марковская модель 2-го порядка': probs_mark_2,
    'Марковская модель 3-го порядка': probs_mark_3,
}

In [ ]:
df_probs = pd.DataFrame(dict_probs)

In [ ]:
df_probs.head()

,Бернулли,Марковская модель 1-го порядка,Марковская модель 2-го порядка,Марковская модель 3-го порядка
0,-85.646165,-69.647092,-67.027975,-66.629843
1,-109.002167,-90.321386,-89.567197,-88.168429
2,-35.863339,-30.309409,-28.277493,-26.181739
3,-100.429481,-84.839577,-83.259203,-79.781595
4,-294.467811,-264.895845,-265.216007,-264.025978


In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go


fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Бернулли",
                                                      "Марковская модель 1-го порядка",
                                                      "Марковская модель 2-го порядка",
                                                      "Марковская модель 3-го порядка"))

bin_numb = len(test_)

# Assuming test_probs, probs_mark_1, probs_mark_2, and probs_mark_3 are lists of probabilities
# and you want to plot them as bar charts. You might need to adjust the x and y values accordingly.

fig.add_trace(go.Bar(x=df_probs.index, y=df_probs['Бернулли'], marker=dict(color='#EDC345'),
                     name='Бернулли'), row=1, col=1)
fig.add_trace(go.Bar(x=df_probs['Марковская модель 1-го порядка'], y=df_probs.index,
                     marker=dict(color='#ED4562'),
                     name='Марковская модель 1-го порядка'), row=1, col=2)
fig.add_trace(go.Bar(x=df_probs['Марковская модель 2-го порядка'], y=df_probs.index, marker=dict(color='#45EDC0'),
                    name='Марковская модель 2-го порядка'), row=2, col=1)
fig.add_trace(go.Bar(x=df_probs['Марковская модель 3-го порядка'], y=df_probs.index, marker=dict(color='#6745ED'),
                     name='Марковская модель 3-го порядка'), row=2, col=2)

fig.update_layout(
    title='Распределения Логарифмических Вероятностей',
    bargap=0.2,
    template='none',
    autosize=True,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()